# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [32]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [33]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#  for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [34]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [35]:
# number of rows in  csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [36]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [37]:
# Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [38]:
# Set KEYSPACE 
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [39]:
## Query to Give  the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

Query1 = "CREATE TABLE IF NOT EXISTS song_details"
Query1 = Query1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                )""")
try:
    session.execute(Query1)
except Exception as e:
    print(e)
    


In [43]:
#  CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statement into the `query` variable
        query = "INSERT INTO song_details(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [44]:

#select statement to get the results
select_query_1 = """ SELECT artist, song_title, song_length FROM song_details
                    WHERE session_id = 338
                    AND item_in_session = 4 
                    """
try:
    rows = session.execute(select_query_1)
except Exception as e:
    print(e)
    


In [45]:
#data output
df=df = pd.DataFrame(rows._current_rows)
print(df)

      artist                       song_title  song_length
0  Faithless  Music Matters (Mark Knight Dub)   495.307312


### Query 2


In [26]:
## Query to Give  only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
Query2 = "CREATE TABLE IF NOT EXISTS artist_details"
Query2 = Query2 + (""" (user_id INT, session_id INT, item_in_session INT, 
                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, 
                                        user_last_name TEXT, PRIMARY KEY ((user_id,session_id ), item_in_session)
                )""")
try:
    session.execute(Query2)
except Exception as e:
    print(e)
    


                    

In [27]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                                                                             
                    

In [47]:
## SELECT statement for the query2
select_query_2 = """ SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_details
                    WHERE user_id = 10
                    AND session_id = 182 
                    """        

try:
    rows = session.execute(select_query_2)
except Exception as e:
    print(e)
    


In [1]:
#output
df=df = pd.DataFrame(rows._current_rows)
print(df)

NameError: name 'pd' is not defined

## Query 3

In [49]:
## Query to Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Query3 = "CREATE TABLE IF NOT EXISTS user_details"
Query3 = Query3 + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, 
                                        PRIMARY KEY ((song_title), user_id)
                )""")
try:
    session.execute(Query3)
except Exception as e:
    print(e)

In [50]:
# Insertion 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [51]:
## SELECT statement for query 3
select_query_3 = """ SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;"""
print(select_query_3)

 SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;


In [52]:
try:
    rows = session.execute(select_query_3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

In [53]:
#output

df=df = pd.DataFrame(rows._current_rows)
print(df)

  user_first_name user_last_name
0      Jacqueline          Lynch
1           Tegan         Levine
2            Sara        Johnson


### Drop the tables before closing out the sessions

In [ ]:
##  Drop the tables 

In [ ]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS artist_details")
session.execute("DROP TABLE IF EXISTS user_details")

### Close the session and cluster connection

In [31]:
#close the session
session.shutdown()
cluster.shutdown()